<table style="border: none" align="left">
   <tr style="border: none">
      <th style="border: none"><font face="verdana" size="5" color="black"><b>Best heart drug prediction using Watson Machine Learning</b></font></th>
      <th style="border: none"><img src="https://github.com/pmservice/customer-satisfaction-prediction/blob/master/app/static/images/ml_icon_gray.png?raw=true" alt="Watson Machine Learning icon" height="40" width="40"></th>
   </tr> 
   <tr style="border: none">
       <td style="border: none"><img src="https://github.com/pmservice/wml-sample-models/raw/master/spark/drug-selection/images/learning_banner-05.png" width="600" alt="Icon"></td>
   </tr>
</table>

This notebook contains steps and code to train, deploy, and start scoring new data. This notebook introduces commands for getting data, model persistance to Watson Machine Learning repository, model deployment, and scoring.

Some familiarity with Python is helpful. This notebook uses Python 3 and Apache Spark 2.1.

You will use the data set published on git, **drug_feedback_data.csv**, which contains anonymous information about patients records. Use the details of this data set to predict the best drug to treat heart disease.

## Learning goals

This notebook teaches you how to:
-  Publish a sample model in the Watson Machine Learning (WML) repository

You will also learn how to use the WML API to:
-  Deploy a model for online scoring 


## Contents

This notebook contains the following parts:

1.	[Set up the environment](#setup)
2.	[Create spark ml model](#model)
3.	[Persist model](#load)
4.	[Deploy & score](#score)
5.	[Summary and next steps](#summary)

<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a [Watson Machine Learning (WML) Service](https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/) instance (a free plan is offered and information about how to create the instance is [here](https://dataplatform.ibm.com/docs/content/analyze-data/wml-setup.html))
- Create a [Spark Service](https://console.ng.bluemix.net/catalog/services/spark/) instance (an entry plan is offered).
- Create a [Db2 Warehouse on Cloud Service](https://console.bluemix.net/catalog/services/db2-warehouse-on-cloud/) instance (an entry plan is offered).
- Create the **DRUG_TRAIN_DATA_UPDATED** table in **Db2 Warehouse on Cloud**. 
  + Download [drug_train_data_updated.csv](https://raw.githubusercontent.com/pmservice/wml-sample-models/master/spark/drug-selection/data/drug_train_data_updated.csv) file from git repository.
  + Click **Open the console** to get started with **Db2 Warehouse on Cloud** icon.
  + Select the **Load Data** and **Desktop** load type.
  + **Drag and drop** previously downloaded file and press **Next**.
  + Select **Schema** to import data and click **New Table**. 
  + Write the name **DRUG_TRAIN_DATA_UPDATED** for **new table** than click **Next** to finish data import.
  + Use `;` as **field separator**.
  + Click **Next** to create a table with the uploaded data.

<a id="model"></a>
## 2. Create the spark machine learning model

In this section you will learn how to prepare data, create an Apache Spark machine learning pipeline, and train a model.

- [2.1 Load the training data from Db2 Warehouse on Cloud](#load)
- [2.2 Prepare the data](#prep)
- [2.3 Create the pipeline](#pipe)
- [2.4 Train the model](#train)

### 2.1 Load the training data from Db2 Warehouse on Cloud<a id="load"></a>

Run the following cell to the load the DRUG_TRAIN_DATA_UPDATED table content into the Spark DataFrame.

Enter your authentication data as required. 

**Tip:** The authentication information can be found under the **Service Credentials**  tab of Db2 Warehouse on Cloud service instance created in IBM Cloud. Click **New credential** to create credentials if you do not have any.

In [ ]:
db2_service_credentials = {
  "port": 50000,
  "db": "BLUDB",
  "username": "*****",
  "ssljdbcurl": "jdbc:db2://dashdb-entry-yp-dal10-01.services.dal.bluemix.net:50001/BLUDB:sslConnection=true;",
  "host": "dashdb-entry-yp-dal10-01.services.dal.bluemix.net",
  "https_url": "https://dashdb-entry-yp-dal10-01.services.dal.bluemix.net:8443",
  "dsn": "***",
  "hostname": "dashdb-entry-yp-dal10-01.services.dal.bluemix.net",
  "jdbcurl": "***",
  "ssldsn": "***",
  "uri": "***",
  "password": "***"
}

In [ ]:
db2_credentials = {
    'driver': 'com.ibm.db2.jcc.DB2Driver',
    'jdbcurl': db2_service_credentials['jdbcurl'],
    'user': db2_service_credentials['username'],
    'password': db2_service_credentials['password']
}

In [ ]:
tablename = "{schema}.{table}".format(schema=db2_credentials['user'], table='DRUG_TRAIN_DATA_UPDATED')

In [ ]:
DRUG_TRAIN_DATA_UPDATED_data = spark.read.jdbc(db2_credentials['jdbcurl'], table=tablename, properties=db2_credentials)

In [ ]:
DRUG_TRAIN_DATA_UPDATED_data.show(5)

The DRUG column is the target/label column.

### 2.2 Prepare the data<a id="prep"></a>

In this subsection you will split your data into two data sets: 
- Train data set
- Test data set

In [ ]:
(train_data, test_data) = DRUG_TRAIN_DATA_UPDATED_data.randomSplit([0.8, 0.2], 24)

print("Number of records for training: " + str(train_data.count()))
print("Number of records for evaluation: " + str(test_data.count()))

As you can see, your data has been successfully split into two data sets:
 - The train data set, which is the largest group, is used for training.
 - The test data set is used for model evaluation.

### 2.3 Create the pipeline<a id="pipe"></a>

In this section you will create an Apache Spark machine learning pipeline.

First, import the Apache Spark machine learning packages that will be needed in the subsequent steps.

In [ ]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, IndexToString, VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline, Model

In the following step, use the StringIndexer transformer to convert all the string fields to numeric ones.

In [ ]:
stringIndexer_sex = StringIndexer(inputCol = 'SEX', outputCol = 'SEX_IX')
stringIndexer_bp = StringIndexer(inputCol = 'BP', outputCol = 'BP_IX')
stringIndexer_chol = StringIndexer(inputCol = 'CHOLESTEROL', outputCol = 'CHOL_IX')
stringIndexer_label = StringIndexer(inputCol="DRUG", outputCol="label").fit(DRUG_TRAIN_DATA_UPDATED_data)

Create a feature vector by combining all the features together.

In [ ]:
vectorAssembler_features = VectorAssembler(inputCols=["AGE", "SEX_IX", "BP_IX", "CHOL_IX", "NA", "K"], outputCol="features")

Next, define the estimators you want to use for classification. Decision Tree is used in the following example.

In [ ]:
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features")

Finally, convert the indexed labels back to the original labels.

In [ ]:
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=stringIndexer_label.labels)

Build the pipeline. A pipeline consists of transformers and an estimator.

In [ ]:
pipeline_dt = Pipeline(stages=[stringIndexer_label, stringIndexer_sex, stringIndexer_bp, stringIndexer_chol, vectorAssembler_features, dt, labelConverter])

### 2.4 Train the model<a id="train"></a>

Now, you can train your Decision Tree model by using the previously defined pipeline and train data.

In [ ]:
model = pipeline_dt.fit(train_data)

You can check your model accuracy now. Use test data to evaluate the model.

In [ ]:
predictions = model.transform(test_data)
evaluatorDT = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluatorDT.evaluate(predictions)

print("Accuracy = %g" % accuracy)

You can tune your model now to achieve better accuracy. To keep this example simple, the tuning section is omitted.

<a id="load"></a>
## 3. Store the model

In this section you will learn how to store sample model in Watson Machine Learning repository by using repository client.

First, install and import the client library.

In [ ]:
!rm -rf $PIP_BUILD/watson-machine-learning-client

In [ ]:
!pip install watson-machine-learning-client --upgrade

**Note**: Apache Spark 2.1 is required.

In [ ]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient
import json

Authenticate to the Watson Machine Learning service on IBM Cloud.

**Tip**: Authentication information (your credentials) can be found in the <a href="https://console.bluemix.net/docs/services/service_credentials.html#service_credentials" target="_blank" rel="noopener no referrer">Service Credentials</a> tab of the service instance that you created on IBM Cloud. 

If you cannot see the **instance_id** field in **Service Credentials**, click **New credential (+)** to generate new authentication information. 

**Action**: Enter your Watson Machine Learning service instance credentials here.


In [ ]:
wml_credentials={
  "url": "https://us-south.ml.cloud.ibm.com",
  "username": "***",
  "password": "***",
  "instance_id": "***"
}

Create the WatsonMachineLearningAPIClient.

In [ ]:
client = WatsonMachineLearningAPIClient(wml_credentials)

#### Prepare the metadata

**Tip**: If the accuracy value falls below the threshold value, retraining action is required.

Prepare the additional information to be saved as model's metadata:
* TRAINING_DATA_REFERENCE
* OUTPUT_DATA_SCHEMA
* EVALUATION_METHOD: **multiclass**
* EVALUATION_METRICS name: **accuracy** (metric name used to evaluate the model)
* EVALUATION_METRICS value: **0.87** (accuracy value calculated few steps above)
* EVALUATION_METRICS threshold: **0.8** (if the accuracy after evaluation using feedback data is below this threshold auto-retraining is triggered)

**Tip**: All required fields can be found on Service Credentials tab of Db2 Warehouse on Cloud service instance created in IBM Cloud.

In [ ]:
training_data_reference = {
 "name": "DRUG feedback",
 "connection": db2_service_credentials,
 "source": {
  "tablename": "DRUG_TRAIN_DATA_UPDATED",
  "type": "dashdb"
 }
}

Define OUTPUT_DATA_SCHEMA

In [ ]:
train_data_schema = train_data.schema
label_field = next(f for f in train_data_schema.fields if f.name == "DRUG")
label_field.metadata['values'] = stringIndexer_label.labels

Set up modelling roles in OUTPUT_DATA_SCHEMA

In [ ]:
from pyspark.sql.types import *

input_fileds = filter(lambda f: f.name != "DRUG", train_data_schema.fields)

output_data_schema = StructType(list(input_fileds)). \
    add("prediction", DoubleType(), True, {'modeling_role': 'prediction'}). \
    add("predictedLabel", StringType(), True, {'modeling_role': 'decoded-target', 'values': stringIndexer_label.labels}). \
    add("probability", ArrayType(DoubleType()), True, {'modeling_role': 'probability'})

Add all the information to model meta props.

In [ ]:
model_props = {
    client.repository.ModelMetaNames.NAME: "drug-selection",
    client.repository.ModelMetaNames.TRAINING_DATA_REFERENCE: training_data_reference,
    client.repository.ModelMetaNames.OUTPUT_DATA_SCHEMA: output_data_schema.jsonValue(),
    client.repository.ModelMetaNames.EVALUATION_METHOD: "multiclass",
    client.repository.ModelMetaNames.EVALUATION_METRICS: [
        {
           "name": "accuracy",
           "value": accuracy,
           "threshold": 0.8
        }
    ]
}

Store the model.

In [ ]:
published_model_details = client.repository.store_model(model=model, meta_props=model_props, training_data=train_data, pipeline=pipeline_dt)
model_uid = client.repository.get_model_uid(published_model_details)

**Tip**: Use `client.repository.ModelMetaNames.show()` to get the list of available props.

<a id="score"></a>
## 4. Deploy and score

You can udeploy previously stored model as web service.

In [ ]:
deployment_details = client.deployments.create(model_uid, 'best-drug model deployment')

In next step, get the scoring endpoint.

In [ ]:
scoring_endpoint = client.deployments.get_scoring_url(deployment_details)
print(scoring_endpoint)

In [ ]:
scoring_payload = {
    "fields": ["AGE", "SEX", "BP", "CHOLESTEROL","NA","K"],
    "values": [[20.0, "F", "HIGH", "HIGH", 0.71, 0.07], [55.0, "M", "LOW", "HIGH", 0.71, 0.07]]
}

Score the model using sample scoring records and scoring_enpoint.

In [ ]:
score = client.deployments.score(scoring_endpoint, scoring_payload)

print(str(score))

<a id="summary"></a>
## 5. Summary and next steps     

You successfully completed this notebook! 
 
You learned how to serve trained model. 
Check out our next notebook: [Data Mart configuration and usage with ibm-ai-openscale python package](https://github.com/pmservice/ai-openscale-sample-notebooks/blob/master/Data%20Mart%20configuration%20and%20usage.ipynb).

### Authors

**Lukasz Cmielowski**, PhD, is an Automation Architect and Data Scientist at IBM with a track record of developing enterprise-level applications that substantially increases clients' ability to turn data into actionable knowledge.

**Maria Oleszkiewicz**, MSc, is a developer who took part in building the wml api client used in this notebook.

Copyright © 2018 IBM. This notebook and its source code are released under the terms of the MIT License.

<div style="background:#F5F7FA; height:110px; padding: 2em; font-size:14px;">
<span style="font-size:18px;color:#152935;">Love this notebook? </span>
<span style="font-size:15px;color:#152935;float:right;margin-right:40px;">Don't have an account yet?</span><br>
<span style="color:#5A6872;">Share it with your colleagues and help them discover the power of Watson Studio!</span>
<span style="border: 1px solid #3d70b2;padding:8px;float:right;margin-right:40px; color:#3d70b2;"><a href="https://ibm.co/wsnotebooks" target="_blank" style="color: #3d70b2;text-decoration: none;">Sign Up</a></span><br>
</div>